# Problème du Monty Hall
### Votre parole contre celle de la machine
<br/>
Ce jeu imaginaire bien connu dans le monde de la statistique en laisse plus d'un perplexe. Et comme l'expliquer intuitivement n'est pas si simple, je vais laisser un algorithme parler pour moi. 
<br/><br/>
<b>Le principe du jeu est le suivant :</b>
<br/>
Lors d'un jeu télévisé, le présentateur montre trois portes à un candidat. Derrière l'une de ces trois portes se trouve une récompense, derrière les deux autres rien. <br/>
<ol> <li>Tout d'abord, le candidat choisit une porte (disons la <b>porte&nbsp;3</b>).</li>
    <li>Puis, le présentateur (qui connaît l'emplacement de la récompense) ouvre l'une des portes, qui n'est ni celle choisie par le candidat, ni celle de la récompense (disons la <b>porte&nbsp;1</b>).</li>
    <li>Enfin, le présentateur permet au candidat de changer sa sélection s'il le souhaite (ici, il peut décider d'abandonner la <b>porte&nbsp;1</b> pour choisir la <b>porte&nbsp;2</b>).</li>
</ol>
La question que pose alors le problème est : le candidat a-t-il intérêt à changer de porte ?
<br/><br/>
Intuitivement, il est tentant de se dire que comme la <b>porte&nbsp;1</b> est écartée par le présentateur, il ne reste que deux portes : la <b>porte&nbsp;2</b> et la <b>porte&nbsp;3</b>. Donc... il a 1 chance sur 2  de gagner ? À quoi bon changer de porte ?
<br/><br/>
<i>Créons un algorithme basique qui tranchera la question. Commençons par un objet qui matérialisera le jeu :</i>

In [145]:
import numpy as np
class MontyHallGame:
    
    def __init__(self, selection):
        self.nb_portes = 3
        
        #on présuppose que la sélection n'entre pas en conflit avec le nombre de portes
        self.selection = selection
        
        #l'algo choisit au hasard la porte gagnante
        self.bonne_porte = np.random.randint(0, self.nb_portes)
        self.porte_ouverte = -1
        
        #print("Nombre de portes : {}".format(self.nb_portes))
        #print("Porte choisie : {}/{}".format(self.selection+1,self.nb_portes))
        
        #portes = [False for _ in np.arange(self.nb_portes)]
        #portes[self.bonne_porte] = True
        #print("Portes : {}".format(portes))
        
        
    #Ouverture d'une porte qui n'est ni gagnante, ni déjà sélectionnée    
    def ouvrir_porte(self):
        #On commence par 0 et croissant ou 3 et décroissant
        #pour éviter de toujours ouvrir la porte 1 (si possible)
        if np.random.randint(2):
            tableau = np.arange(2,-1,-1)
        else:
            tableau = np.arange(3)
        for i in tableau:
            if i != self.selection and i != self.bonne_porte:
                self.porte_ouverte = i
                #print("Le présentateur ouvre la porte : {}".format(self.porte_ouverte+1))
                return i
        
    #Le candidat peut changer de porte    
    def changer_porte(self):
        for i in np.arange(self.nb_portes):
            if (i == self.porte_ouverte) or (i == self.selection):
                continue
            self.selection = i
            break
        #print("Nouvelle porte choisie : {}/{}".format(self.selection+1,self.nb_portes))
    
    #Le candidat a-t-il gagné ?
    def resultat(self):
        #print("La porte sélectionnée est correcte : {}\n".format(self.selection == self.bonne_porte))
        return self.selection == self.bonne_porte      
        

<i>Procédons ensuite à un nombre d'essais suffisamment grand pour être significatif. Dans les 10 000 premiers essais, le candidat <b>ne change pas de porte</b>. Dans les 10 000 suivants, <b>il change systématiquement</b>.</i>

In [146]:
print("--- Jeu du Monty Hall ---")
print("--- En deux fois 10 000 essais ---\n")
print("\t--- Sans changer de porte :")
nb_essais = 10000
nb_reussites = 0
for _ in np.arange(0, nb_essais):
    porte = np.random.randint(0, 3)        
    a = MontyHallGame(porte)
    a.ouvrir_porte()       
    if a.resultat():
        nb_reussites += 1
        
print("\t{} réussites sur {}, soit environ {}% de succès ---\n"
      .format(nb_reussites, nb_essais, 100*nb_reussites/nb_essais )) 

#réinitialisation du nombre de réussites
nb_reussites = 0
print("\t--- En changeant de porte :")
for _ in np.arange(0, nb_essais):
    porte = np.random.randint(0, 3)        
    a = MontyHallGame(porte)
    a.ouvrir_porte() 
    a.changer_porte()
    if a.resultat():
        nb_reussites += 1
        
print("\t{} réussites sur {}, soit environ {}% de succès ---"
      .format(nb_reussites, nb_essais, nb_reussites/nb_essais*100 )) 


--- Jeu du Monty Hall ---
--- En deux fois 10 000 essais ---

	--- Sans changer de porte :
	3417 réussites sur 10000, soit environ 34.17% de succès ---

	--- En changeant de porte :
	6697 réussites sur 10000, soit environ 66.97% de succès ---


Pour ceux qui connaissent le problème, le résultat n'est pas étonnant : le candidat a tout intérêt à changer de porte, car il a alors <b>deux chances sur trois</b> de gagner la récompense. Comment l'expliquer ? Tout vient du fait que le présentateur connaît le bon emplacement et n'a pas dévoilé sa porte <i>par hasard</i>.
<br/><br/>
Il est plus intuitif de commencer par se dire qu'au départ, le candidat a 1 chance sur 3 de gagner : il fait <b>un</b> choix, il y a <b>trois</b> portes. S'il ne change pas de porte, cette probabilité n'a pas de raison de changer non plus.
<br/>
À partir de là, si le candidat pouvait décider de choisir <i><b>tout ce qui n'est pas son premier choix</b></i>, il aurait 1 - 1/3 = 2/3 chances de gagner.
<br/>
Et effectivement, comme le présentateur a éliminé une porte, <i><b>l'ensemble des choix possibles qui ne sont pas le premier choix</b></i> du candidat... c'est la porte non dévoilée.
<br/><br/>Si vous avez vent d'une méthode encore plus convaincante pour faire briller cette lueur de lucidité chez l'auditeur, elle est la bienvenue.<br/><br/>
<center>À bientôt pour un nouvel épisode de <b>Fun with Stats</b> !</center>